In [13]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.precision', 2)
import json

In [14]:
results_path = os.path.join("..","trained_models")
experiments_name = [folder for folder in os.listdir(results_path) 
                    if os.path.isdir(os.path.join(results_path, folder))]

experiments_dict = {
    'datetime':[],
    'dataset':[],
    'model':[],
    'distribution_head':[],
    'scaling':[],
    'model_params':[],
    'epochs':[],
    'early_stop':[],
    'actual':[],
    'forecast':[],
    'metrics':[]
}

baselines = ['iETS', 'ZeroForecast', 'NaiveW']

for exp in experiments_name:
    skip = False
    for b in baselines:
        if b in exp: skip = True
    exp_split = exp.split('__')
    experiments_dict['dataset'].append( exp_split[1] )
    experiments_dict['model'].append( exp_split[0] )
    experiments_dict['actual'].append( os.path.join(results_path,exp,'actuals.npy') )
    experiments_dict['forecast'].append( os.path.join(results_path,exp,'forecasts.npy') )
    experiments_dict['metrics'].append( json.load(open(os.path.join(results_path,exp,'metrics.json'), "r")) )
    experiments_dict['datetime'].append( exp_split[2] )
    if skip:
        experiments_dict['distribution_head'].append( "-" )
        experiments_dict['scaling'].append( '-' )
        experiments_dict['epochs'].append( np.nan )
        experiments_dict['early_stop'].append( np.nan )
        experiments_dict['model_params'].append( np.nan )
        continue
    exp_info = json.load(open(os.path.join(results_path,exp,'experiment.json'), "r"))
    experiments_dict['distribution_head'].append( exp_info['distribution_head'] )
    experiments_dict['scaling'].append( exp_info['scaling'] if exp_info['scaling'] else '-' )
    experiments_dict['epochs'].append( exp_info['epoch'] )
    experiments_dict['early_stop'].append( exp_info['early_stop'] )
    experiments_dict['model_params'].append( json.load(open(os.path.join(results_path,exp,'model_params.json'), "r")) )

In [15]:
experiments_df = pd.DataFrame(experiments_dict)
experiments_df['datetime'] = pd.to_datetime(experiments_df['datetime'], format="%Y-%m-%d-%H:%M:%S:%f")
# -- 
experiments_df = experiments_df[(experiments_df.dataset == "carparts") & ((experiments_df.model == "deepAR") | experiments_df.model.isin(baselines))]
experiments_df.sort_values('datetime', inplace=True)
# --
experiments_df.set_index(['datetime','dataset','model','distribution_head','scaling'], inplace=True)

quantile_losses = pd.DataFrame([values['metrics']['quantile_loss'] for _, values in experiments_df.iterrows()])
quantile_losses.set_index(experiments_df.index, inplace=True)
styled_quantile_losses = quantile_losses.style.apply(lambda x: ['background-color: green' if v == x.min() else '' for v in x])

styled_quantile_losses